In [1]:
import math
import numpy as np
import random
from collections import Counter
import pandas as pd

def generate_sample(courses,hours_per_course,max_lecture_times, rooms_num, weekdays_num, max_lecture_hours): 
    #generate (room, weekday) combination for 9 times
    population = [(room, weekday) for room in range(rooms_num) for weekday in range(weekdays_num) for _ in range(int(max_lecture_hours))]
    
    #initialization timetable
    #course:[(room,weekday)]
    timetable={key:None for key in courses}
    num=0  # Variable to count the total number of scheduled hours
    for i in range(len(courses)):
        pick_hour=hours_per_course[courses[i]] #duration of course
        s_list=[] #sampling list for a course
        for k in range(max_lecture_times[courses[i]]): #worst assignment, greatest number of class for all erolled students
            count_dict = dict(Counter(population)) #(room,weekday):left hours, only available
            count_dict={key: value for key, value in count_dict.items() if value >= pick_hour}# Filter slots that can still accommodate the hours needed for the current course.
            temp_population=[key for key, value in count_dict.items() for _ in range(value)]   # Weight each time slot by its remaining capacity, increasing the chance of being chosen.
            s = random.sample(temp_population, 1)[0] # Randomly select a time slot.
            
            s_list.append(s)# Append the selected slot to the course's list of slots.
            for j in range(pick_hour):# Remove the allocated slot from the population.
                num+=1
                population.remove(s)
        timetable[courses[i]]=s_list# Update the course's list of allocated time slots.
    print("num:",num)
    return timetable,population

def mutation(timetable, courses, variation, unoccupied_time, hours_per_course):
    # Start mutation process by selecting a specific course based on a variation index.
    course = courses[variation]
    # Retrieve the number of hours needed for the course.
    hours = hours_per_course[course]
    # Randomly select a current time slot index for this course to mutate.
    index = random.randint(0, len(timetable[course]) - 1)
    # Count available slots in unoccupied_time that can accommodate the course duration.
    count_dict = dict(Counter(unoccupied_time))
    count_dict = {key: value for key, value in count_dict.items() if value >= hours}
    temp_population = [key for key, value in count_dict.items() for _ in range(value)]
    index_unoccupied = random.randint(0, len(temp_population) - 1)
    
    # Perform the swap of the selected slot with an unoccupied slot.
    temp = timetable[course][index]
    before = len(temp)
    timetable[course][index] = unoccupied_time[index_unoccupied]
    unoccupied_time[index_unoccupied] = temp
    # If the course duration is more than 1 hour, additional slots need to be managed.
    if hours > 1:
        for i in range(hours - 1):
            unoccupied_time.remove(temp)
    after = len(temp)
    # Debugging output to verify the process
    return timetable, unoccupied_time
    
def exchange(timetable, courses, variation_1, variation_2, hours_per_course, unoccupied_time):
    # Select two courses based on variation indices for exchange.
    course_1 = courses[variation_1]
    course_2 = courses[variation_2]
    hour_1 = hours_per_course[course_1]
    hour_2 = hours_per_course[course_2]
    # If both courses have the same duration, a simple swap is possible.
    if hour_1 == hour_2:
        index_1 = random.randint(0, len(timetable[course_1]) - 1)
        index_2 = random.randint(0, len(timetable[course_2]) - 1)
        temp = timetable[course_1][index_1]
        timetable[course_1][index_1] = timetable[course_2][index_2]
        timetable[course_2][index_2] = temp
    else:
        # More complex exchange logic when courses have different durations.
        index_2 = random.randint(0, len(timetable[course_2]) - 1)
        index_1 = random.randint(0, len(timetable[course_1]) - 1)
        rw_1 = timetable[course_1][index_1]
        rw_2 = timetable[course_2][index_2]
        if rw_1 in unoccupied_time:
            index_unoccupied_1 = unoccupied_time.index(rw_1)
        else:
            index_unoccupied_1 = -1
        if rw_2 in unoccupied_time:
            index_unoccupied_2 = unoccupied_time.index(rw_2)
        else:
            index_unoccupied_2 = -1

        # Exchange slots if appropriate conditions are met.
        if hour_1 > hour_2 and index_unoccupied_2 > -1:
            temp = timetable[course_1][index_1]
            timetable[course_1][index_1] = timetable[course_2][index_2]
            timetable[course_2][index_2] = temp
            unoccupied_time.append(temp)
            unoccupied_time.remove(rw_2)
        if hour_1 < hour_2 and index_unoccupied_1 > -1:
            temp = timetable[course_2][index_2]
            timetable[course_2][index_2] = timetable[course_1][index_1]
            timetable[course_1][index_1] = temp
            unoccupied_time.append(temp)
            unoccupied_time.remove(rw_1)
    return timetable, unoccupied_time


def count_lecture_num(lecture_arrangement, students_per_course, course_str, rooms, room_capacities):
    # Start with the total number of students enrolled in the specified course.
    student_left = students_per_course[course_str]
    # Initialize the count of lectures needed to accommodate all students.
    lecture_num = 0
    
    # Iterate through each lecture in the arrangement for the course.
    for lecture in lecture_arrangement:
        # If there are no students left to accommodate, stop the loop.
        if student_left <= 0:
            break
        # Get the room number from the lecture tuple.
        room_num = lecture[0]
        # Find the capacity of the room using the room number.
        room_cap = room_capacities[rooms[room_num]]
        
        # If the remaining students exceed the room's capacity, subtract the capacity from the remaining students.
        if student_left >= room_cap:
            student_left -= room_cap
            lecture_num += 1
            # Output the room and its capacity.
            print(rooms[room_num], ":", room_cap)
        else:
            # If fewer students remain than the room capacity, just account for them.
            if student_left > 0:
                lecture_num += 1
                # Output the room and the number of remaining students.
                print(rooms[room_num], ":", student_left)
                student_left = 0
    # Print the total number of lectures required for the course.
    print(course_str, "number_of_class:", lecture_num)
    return lecture_num

def total_classes_num(courses, timetable, students_per_course, rooms, room_capacities):
    # Initialize the total number of classes across all courses.
    total_num = 0
    
    # Iterate through each course in the list.
    for i in range(len(courses)):
        course_str = courses[i]
        # Retrieve the specific timetable arrangement for the course.
        lecture_arrangement = timetable[course_str]
        # Count the number of classes needed for the course.
        lec_num = count_lecture_num(lecture_arrangement=lecture_arrangement,
                                    students_per_course=students_per_course,
                                    course_str=course_str,
                                    rooms=rooms,
                                    room_capacities=room_capacities)
        # Add the number of classes for this course to the total.
        total_num += lec_num
    # Return the total number of classes for all courses.
    return total_num


In [2]:
courses = [
    'MATH08062', 'MATH10066', 'MATH11231', 'MATH08077',
    'MATH08057', 'MATH10098', 'MATH11199', 'MATH11029',
    'MATH10099', 'MATH11029+MATH11197', 'MATH10098+MATH10066',
    'MATH10068+MATH08068+MATH10079',
    
    'MATH08071', 'MATH11226', 'MATH11236', 'MATH11177', 'MATH10072', 'MATH10017',
    'MATH11235', 'MATH11153', 'MATH08074', 'MATH10047', 'MATH10065', 'MATH11111',
    'MATH10076', 'MATH11187', 'MATH10074',
    'MATH11053', 'MATH10100', 'MATH10082', 'MATH10013', 'MATH08072',
    'MATH11007', 'MATH08066', 'MATH10024', 'MATH08063',
    'MATH10095', 'MATH11176', 'MATH11154', 'MATH10007', 'MATH10028',
    'MATH11144', 'MATH11179', 'MATH10066A', 
    'MATH11236 + MATH11177', 'MATH11230 + MATH11153', 'MATH10072 + MATH10047',
    'MATH10017 + MATH10065', 'MATH11235 + MATH10076', 'MATH11111 + MATH11187',
    'MATH11226 + MATH10053', 'MATH10082 + MATH11053', 'MATH08066 + MATH10100',
    'MATH10013 + MATH10074', 'MATH08074 + MATH10095', 'MATH08068 + MATH08074',
    'MATH11144 + MATH11179', 'MATH07003 + MATH11154', 'MATH10010 + MATH10053',
    'MATH10098 + MATH10028', 'MATH10024 + MATH10079','MATH10066', 'MATH11231A', 
    'MATH08077A', 'MATH08057A', 'MATH10098A',
    'MATH11199A', 'MATH11029A'
]
students_per_course = {
    'MATH08071': 71, 'MATH11226': 38, 'MATH11236': 22, 'MATH11177': 161, 'MATH10072': 96,
    'MATH10017': 30, 'MATH11235': 35, 'MATH11153': 88, 'MATH08074': 427, 'MATH10047': 25,
    'MATH10065': 216, 'MATH11111': 174, 'MATH10076': 54, 'MATH11187': 138, 'MATH10074': 97,
    'MATH10066A': 308, 'MATH11231A': 32, 'MATH08077A': 180, 'MATH11053': 13, 'MATH08057A': 585,
    'MATH10100': 33, 'MATH10082': 25, 'MATH10013': 60, 'MATH08072': 141, 'MATH11007': 78,
    'MATH10098A': 145, 'MATH08066': 458, 'MATH10024': 7, 'MATH11199A': 228, 'MATH08063': 452,
    'MATH10095': 262, 'MATH11176': 140, 'MATH11154': 79, 'MATH11029A': 59, 'MATH10007': 149,
    'MATH10028': 58, 'MATH11144': 5, 'MATH11179': 29, 'MATH11236 + MATH11177':161,'MATH11230 + MATH11153': 88, 
    'MATH10072 + MATH10047': 96, 'MATH10017 + MATH10065': 216, 'MATH11235 + MATH10076': 54, 
    'MATH11111 + MATH11187': 174, 'MATH11226 + MATH10053': 50, 'MATH10082 + MATH11053': 25, 
    'MATH08066 + MATH10100': 458, 'MATH10013 + MATH10074': 97, 'MATH08074 + MATH10095': 427, 
    'MATH08068 + MATH08074': 427, 'MATH11144 + MATH11179': 29, 'MATH07003 + MATH11154': 248, 
    'MATH10010 + MATH10053': 53, 'MATH10098 + MATH10028': 145, 'MATH10024 + MATH10079': 50,'MATH08062': 57, 'MATH10066': 308, 'MATH11231': 32, 'MATH08077': 180, 'MATH08057': 585,
    'MATH10098': 145, 'MATH11199': 228, 'MATH11029': 59, 'MATH10099': 43,
    'MATH11029+MATH11197': 59, 'MATH10098+MATH10066': 300, 'MATH10068+MATH08068+MATH10079': 274
}
hours_per_course = {
    'MATH08071': 1, 'MATH11226': 1, 'MATH11236': 1, 'MATH11177': 1, 'MATH10072': 1,
    'MATH10017': 1, 'MATH11235': 1, 'MATH11153': 1, 'MATH08074': 1, 'MATH10047': 1,
    'MATH10065': 1, 'MATH11111': 1, 'MATH10076': 1, 'MATH11187': 1, 'MATH10074': 1,
    'MATH10066A': 1, 'MATH11231A': 1, 'MATH08077A': 1, 'MATH11053': 1, 'MATH08057A': 1,
    'MATH10100': 1, 'MATH10082': 1, 'MATH10013': 1, 'MATH08072': 1, 'MATH11007': 1,
    'MATH10098A': 1, 'MATH08066': 1, 'MATH10024': 1, 'MATH11199A': 1, 'MATH08063': 1,
    'MATH10095': 1, 'MATH11176': 1, 'MATH11154': 1, 'MATH11029A': 1, 'MATH10007': 1,
    'MATH10028': 1, 'MATH11144': 1, 'MATH11179': 1, 'MATH11236 + MATH11177':1 ,'MATH11230 + MATH11153': 1,
    'MATH10072 + MATH10047': 1, 'MATH10017 + MATH10065': 1, 'MATH11235 + MATH10076': 1,
    'MATH11111 + MATH11187': 1, 'MATH11226 + MATH10053': 1, 'MATH10082 + MATH11053': 1,
    'MATH08066 + MATH10100': 1, 'MATH10013 + MATH10074': 1, 'MATH08074 + MATH10095': 1,
    'MATH08068 + MATH08074': 1, 'MATH11144 + MATH11179': 1, 'MATH07003 + MATH11154': 1,
    'MATH10010 + MATH10053': 1, 'MATH10098 + MATH10028': 1, 'MATH10024 + MATH10079': 1,
    'MATH08062': 2, 'MATH10066': 2, 'MATH11231': 2, 'MATH08077': 2,
    'MATH08057': 2, 'MATH10098': 2, 'MATH11199': 2, 'MATH11029': 2,
    'MATH10099': 2, 'MATH11029+MATH11197': 2, 'MATH10098+MATH10066': 2,
    'MATH10068+MATH08068+MATH10079': 2
}
 

rooms = [
    'JBB_G.69', 'JCMB_2901', 'JCMB_3211 - Teaching Studio', 'JCMB_3212 - Teaching Studio',
    'JCMB_3217 - Teaching Studio', 'MH_1.19', 'MH_G.04', 'MH_G.05', 'MH_G.10', 'MH_G.12',
    'MH_G.13', 'MH_G.30', 'MH_G.31', 'MH_G.32', 'MH_G.33', 'MH_G.34', 'NUC_2.06 - Rowan',
    'SB_7.15', 'SB_7.20', 'AT_M2 - Teaching Studio', '40GS_LG.07 - Teaching Studio',
    '40GS_LG.09', '40GS_LG.11', 'AT_2.12', 'LLTC_2.14 - Teaching Studio',
    'ROB_Hugh Robson Building Computer Lab - 129 cap', '40GS_7.18', '40GS_7.01'
]  
 

room_capacities = {
    'JBB_G.69': 25, 'JCMB_2901': 14, 'JCMB_3211 - Teaching Studio': 24,
    'JCMB_3212 - Teaching Studio': 24, 'JCMB_3217 - Teaching Studio': 84,
    'MH_1.19': 68, 'MH_G.04': 20, 'MH_G.05': 24, 'MH_G.10': 20,
    'MH_G.12': 20, 'MH_G.13': 19, 'MH_G.30': 20, 'MH_G.31': 46,
    'MH_G.32': 28, 'MH_G.33': 20, 'MH_G.34': 24, 'NUC_2.06 - Rowan': 60,
    'SB_7.15': 50, 'SB_7.20': 58, 'AT_M2 - Teaching Studio': 25,
    '40GS_LG.07 - Teaching Studio': 20, '40GS_LG.09': 90, '40GS_LG.11': 60,
    'AT_2.12': 20, 'LLTC_2.14 - Teaching Studio': 36,
    'ROB_Hugh Robson Building Computer Lab - 129 cap': 12, '40GS_7.18': 18, '40GS_7.01': 12
} 

In [3]:
max_lecture_hours=9# Maximum number of lecture hours that can be scheduled in a single day

weekdays_num = 5# Total number of weekdays for scheduling (Monday to Friday)

mutation_time=10#number of new timetable

total_mutation=500#mutation and exchange
date={0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday'}

In [4]:

# Determine the total number of rooms available based on the length of the 'rooms' list or array.
rooms_num = len(rooms)

# Determine the total number of courses based on the length of the 'courses' list or array.
courses_num = len(courses)

# Calculate the minimum capacity among all rooms. This is the smallest number of students that any room can accommodate.
min_cap_room = min(room_capacities.values())

# Calculate the maximum number of lecture times for each course. This is determined by dividing the total number of students per course
# by the minimum capacity room and rounding up. This gives the minimum number of sessions needed for each course if only the smallest room were used.
max_lecture_times = {key: math.ceil(value / min_cap_room) for key, value in students_per_course.items()}

# Generate a sample timetable and a list of unoccupied time slots. This involves creating a (room, weekday) combination for each available room
# and weekday for the maximum number of lecture hours specified.
timetable, unoccupied_time = generate_sample(courses=courses, hours_per_course=hours_per_course,
                                             max_lecture_times=max_lecture_times, rooms_num=rooms_num, 
                                             weekdays_num=weekdays_num, max_lecture_hours=max_lecture_hours)


num: 1128


In [5]:
pre_timetable=timetable
pre_timetable

{'MATH08062': [(22, 3), (4, 2), (19, 0), (8, 0), (15, 1)],
 'MATH10066': [(17, 4),
  (1, 4),
  (11, 1),
  (12, 3),
  (2, 0),
  (26, 2),
  (22, 0),
  (3, 1),
  (20, 1),
  (26, 0),
  (8, 4),
  (2, 3),
  (6, 0),
  (2, 2),
  (3, 3),
  (15, 2),
  (4, 1),
  (0, 0),
  (15, 0),
  (5, 1),
  (1, 1),
  (6, 1),
  (2, 0),
  (16, 1),
  (21, 2),
  (10, 1)],
 'MATH11231': [(11, 3), (26, 3), (2, 2)],
 'MATH08077': [(5, 0),
  (10, 0),
  (11, 4),
  (25, 3),
  (7, 1),
  (27, 0),
  (2, 4),
  (6, 4),
  (10, 4),
  (9, 4),
  (25, 3),
  (15, 1),
  (4, 3),
  (2, 3),
  (1, 3)],
 'MATH08057': [(15, 1),
  (22, 1),
  (9, 4),
  (19, 1),
  (22, 3),
  (17, 3),
  (11, 4),
  (11, 0),
  (3, 3),
  (0, 1),
  (12, 0),
  (17, 1),
  (20, 4),
  (23, 1),
  (11, 1),
  (24, 2),
  (8, 0),
  (19, 2),
  (17, 4),
  (24, 1),
  (22, 4),
  (13, 4),
  (16, 2),
  (27, 2),
  (13, 4),
  (7, 2),
  (6, 2),
  (13, 3),
  (1, 3),
  (1, 3),
  (7, 4),
  (26, 4),
  (9, 3),
  (16, 4),
  (5, 1),
  (12, 2),
  (15, 2),
  (27, 1),
  (18, 0),
  (4, 1),
 

In [6]:
# Generate a list of all possible room and weekday combinations.
room_timetable = [(room, weekday) for room in range(rooms_num) for weekday in range(weekdays_num)]

# Create a dictionary to map each room-weekday pair to a list that will contain course data.
room_timetable_dict = {key: [] for key in room_timetable}

# Populate the room_timetable_dict with course and hour information from a predefined timetable.
for course in pre_timetable:
    arrangement_list = pre_timetable[course]
    for room_date in arrangement_list:
        # Each entry in the room's timetable includes the course and the number of hours it takes.
        course_time = (course, hours_per_course[course])
        room_timetable_dict[room_date].append(course_time)

# Iterate through each room-weekday combination to display the scheduled courses.
for rt in room_timetable:
    # Print the room number and the day it's being used.
    print(rooms[rt[0]], ":")
    # Initialize 'k' to calculate the starting time for the first class of the day at 9 AM.
    k = 0
    # Process each class scheduled in this room on this day.
    for one_class in room_timetable_dict[rt]:
        # Prepare the display string with course code and the specific day.
        first_part = "  - Course Code: " + one_class[0] + ", " + "Day " + str(rt[1] + 1) + ", "
        # Calculate starting and ending times for the class based on its duration.
        second_part = [9 + k, 9 + k + one_class[1]]
        # Convert times to a 12-hour format with AM/PM notation.
        for j in range(2):
            if second_part[j] < 12:
                second_part[j] = str(second_part[j]) + "AM"
            elif second_part[j] == 12:
                second_part[j] = "12PM"
            else:
                second_part[j] = str(second_part[j] % 12) + "PM"
        # Append the duration of the class to the display string.
        third_part = "(" + str(one_class[1]) + "h)"
        # Update 'k' with the ending time of the current class to start the next class at the correct time.
        k += one_class[1]
        # Print the formatted information for this class.
        print(first_part + second_part[0] + "-" + second_part[1] + third_part)


JBB_G.69 :
  - Course Code: MATH10066, Day 1, 9AM-11AM(2h)
  - Course Code: MATH08074, Day 1, 11AM-12PM(1h)
  - Course Code: MATH11007, Day 1, 12PM-1PM(1h)
  - Course Code: MATH08066, Day 1, 1PM-2PM(1h)
  - Course Code: MATH10017 + MATH10065, Day 1, 2PM-3PM(1h)
  - Course Code: MATH08068 + MATH08074, Day 1, 3PM-4PM(1h)
  - Course Code: MATH07003 + MATH11154, Day 1, 4PM-5PM(1h)
JBB_G.69 :
  - Course Code: MATH08057, Day 2, 9AM-11AM(2h)
  - Course Code: MATH08074, Day 2, 11AM-12PM(1h)
  - Course Code: MATH10076, Day 2, 12PM-1PM(1h)
  - Course Code: MATH11187, Day 2, 1PM-2PM(1h)
  - Course Code: MATH10017 + MATH10065, Day 2, 2PM-3PM(1h)
  - Course Code: MATH08074 + MATH10095, Day 2, 3PM-4PM(1h)
JBB_G.69 :
  - Course Code: MATH10068+MATH08068+MATH10079, Day 3, 9AM-11AM(2h)
  - Course Code: MATH08066, Day 3, 11AM-12PM(1h)
  - Course Code: MATH10095, Day 3, 12PM-1PM(1h)
  - Course Code: MATH08074 + MATH10095, Day 3, 1PM-2PM(1h)
  - Course Code: MATH08057A, Day 3, 2PM-3PM(1h)
  - Course Code:

In [7]:
# Initialize a list to store the number of classes for each course.
total_classes = []
# Iterate over each course based on the total number of courses.
for i in range(courses_num):
    # Calculate the number of classes needed for each course using the count_lecture_num function.
    class_n = count_lecture_num(
        lecture_arrangement=timetable[courses[i]],
        students_per_course=students_per_course,
        course_str=courses[i],
        rooms=rooms,
        room_capacities=room_capacities
    )
    # Append the result to the list of total classes.
    total_classes.append(class_n)

# Print the total number of classes for each course.
print("total_class:", total_classes)

# Calculate the sum of all classes required for all courses and print it.
sum_total_classes = sum(total_classes)
print("Sum of all classes:", sum_total_classes)



40GS_LG.11 : 57
MATH08062 number_of_class: 1
SB_7.15 : 50
JCMB_2901 : 14
MH_G.30 : 20
MH_G.31 : 46
JCMB_3211 - Teaching Studio : 24
40GS_7.18 : 18
40GS_LG.11 : 60
JCMB_3212 - Teaching Studio : 24
40GS_LG.07 - Teaching Studio : 20
40GS_7.18 : 18
MH_G.10 : 14
MATH10066 number_of_class: 11
MH_G.30 : 20
40GS_7.18 : 12
MATH11231 number_of_class: 2
MH_1.19 : 68
MH_G.13 : 19
MH_G.30 : 20
ROB_Hugh Robson Building Computer Lab - 129 cap : 12
MH_G.05 : 24
40GS_7.01 : 12
JCMB_3211 - Teaching Studio : 24
MH_G.04 : 1
MATH08077 number_of_class: 8
MH_G.34 : 24
40GS_LG.11 : 60
MH_G.12 : 20
AT_M2 - Teaching Studio : 25
40GS_LG.11 : 60
SB_7.15 : 50
MH_G.30 : 20
MH_G.30 : 20
JCMB_3212 - Teaching Studio : 24
JBB_G.69 : 25
MH_G.31 : 46
SB_7.15 : 50
40GS_LG.07 - Teaching Studio : 20
AT_2.12 : 20
MH_G.30 : 20
LLTC_2.14 - Teaching Studio : 36
MH_G.10 : 20
AT_M2 - Teaching Studio : 25
SB_7.15 : 20
MATH08057 number_of_class: 19
JCMB_3211 - Teaching Studio : 24
MH_G.30 : 20
JCMB_3212 - Teaching Studio : 24
NUC_2

In [8]:
# Initialize the total decrease in the number of class sessions.
total_decrease = 0

# Iterate over the total number of mutations specified.
for i in range(total_mutation):
    # Randomly select two courses for potential mutation or exchange.
    variation_1 = random.randint(0, len(courses) - 1)
    variation_2 = random.randint(0, len(courses) - 1)

    if variation_1 == variation_2:
        # If the same course is selected twice, perform a self-mutation.
        print("mutation")
        mutation_list = []
        class_num = []

        # Perform the mutation multiple times (as per mutation_time) and store the results.
        for i in range(mutation_time):
            course_str = courses[variation_1]
            timetable_temp, unoccupied_time_temp = mutation(timetable=timetable,
                                                            courses=courses,
                                                            variation=variation_1,
                                                            unoccupied_time=unoccupied_time,
                                                            hours_per_course=hours_per_course)
            # Count the number of classes for the mutated timetable.
            class_num.append(count_lecture_num(lecture_arrangement=timetable_temp[course_str],
                                               students_per_course=students_per_course,
                                               course_str=course_str,
                                               rooms=rooms,
                                               room_capacities=room_capacities))
            mutation_list.append((timetable_temp, unoccupied_time_temp))

        print("class_num:", class_num)
        # Find the mutation with the minimum number of classes.
        index_ = class_num.index(min(class_num))
        if total_classes[variation_1] > min(class_num):
            # If the mutation reduces the number of classes, update the timetable.
            timetable.update(mutation_list[index_][0])
            unoccupied_time = mutation_list[index_][1]
            print(total_classes[variation_1], "v.s.", min(class_num))
            total_classes[variation_1] = min(class_num)
            print("new self mutation")
            total_decrease += 1
        print("done")

    else:
        # If different courses are selected, perform a gene exchange.
        print("exchange:")
        mutation_list = []
        class_num = []

        # Perform the exchange multiple times and store the results.
        for i in range(mutation_time):
            timetable_temp, unoccupied_time_temp = exchange(timetable=timetable, courses=courses,
                                                            variation_1=variation_1, variation_2=variation_2,
                                                            hours_per_course=hours_per_course, unoccupied_time=unoccupied_time)
            course_str_1 = courses[variation_1]
            course_str_2 = courses[variation_2]
            num_1 = count_lecture_num(lecture_arrangement=timetable_temp[course_str_1],
                                      students_per_course=students_per_course,
                                      course_str=course_str_1,
                                      rooms=rooms,
                                      room_capacities=room_capacities)
            num_2 = count_lecture_num(lecture_arrangement=timetable_temp[course_str_2],
                                      students_per_course=students_per_course,
                                      course_str=course_str_2,
                                      rooms=rooms,
                                      room_capacities=room_capacities)
            class_num.append((num_1, num_2))
            mutation_list.append((timetable_temp, unoccupied_time_temp))

        # Calculate the sum of class sessions for each mutation.
        sum_len = [i + j for (i, j) in class_num]
        index_ = sum_len.index(min(sum_len))
        # If the total class sessions decrease, update the timetable.
        if total_classes[variation_1] + total_classes[variation_2] > min(sum_len):
            timetable.update(mutation_list[index_][0])
            unoccupied_time = mutation_list[index_][1]
            before_times = total_classes[variation_1] + total_classes[variation_2]
            after_times = min(sum_len)
            print(before_times, "v.s.", after_times)
            print("new exchange")
            total_classes[variation_1] = class_num[index_][0]
            total_classes[variation_2] = class_num[index_][1]
            total_decrease += before_times - after_times
        print("done")

# Output the total reduction in class sessions achieved by the mutations.
print("Total decrease in class sessions:", total_decrease)






exchange:
40GS_LG.09 : 25
MATH10082 number_of_class: 1
AT_M2 - Teaching Studio : 25
MH_G.33 : 8
MATH10100 number_of_class: 2
MH_G.33 : 20
JCMB_2901 : 5
MATH10082 number_of_class: 2
AT_M2 - Teaching Studio : 25
40GS_LG.09 : 8
MATH10100 number_of_class: 2
MH_G.33 : 20
JCMB_2901 : 5
MATH10082 number_of_class: 2
MH_G.13 : 19
40GS_LG.09 : 14
MATH10100 number_of_class: 2
MH_G.33 : 20
JCMB_2901 : 5
MATH10082 number_of_class: 2
AT_M2 - Teaching Studio : 25
40GS_LG.09 : 8
MATH10100 number_of_class: 2
MH_G.33 : 20
JCMB_2901 : 5
MATH10082 number_of_class: 2
AT_M2 - Teaching Studio : 25
40GS_LG.09 : 8
MATH10100 number_of_class: 2
MH_G.33 : 20
MH_G.13 : 5
MATH10082 number_of_class: 2
AT_M2 - Teaching Studio : 25
40GS_LG.09 : 8
MATH10100 number_of_class: 2
MH_G.33 : 20
JCMB_2901 : 5
MATH10082 number_of_class: 2
AT_M2 - Teaching Studio : 25
40GS_LG.09 : 8
MATH10100 number_of_class: 2
MH_G.33 : 20
MH_G.13 : 5
MATH10082 number_of_class: 2
AT_M2 - Teaching Studio : 25
40GS_LG.09 : 8
MATH10100 number_of_

In [9]:
print("total_class:",total_classes)#total_class
sum(total_classes)#sum

total_class: [1, 9, 2, 3, 16, 2, 6, 1, 1, 3, 7, 7, 2, 1, 1, 3, 2, 1, 1, 3, 9, 1, 4, 4, 1, 4, 2, 1, 2, 1, 1, 4, 1, 14, 1, 9, 6, 3, 2, 5, 1, 1, 1, 8, 4, 2, 2, 4, 1, 3, 1, 1, 13, 3, 13, 11, 1, 7, 1, 4, 1, 8, 1, 2, 14, 4, 5, 1]


265

In [10]:
timetable

{'MATH08062': [(12, 1), (27, 2), (7, 0), (22, 4), (25, 2)],
 'MATH10066': [(21, 4),
  (3, 0),
  (27, 1),
  (23, 0),
  (20, 1),
  (14, 3),
  (23, 2),
  (19, 2),
  (23, 3),
  (9, 0),
  (16, 1),
  (1, 1),
  (3, 4),
  (0, 4),
  (0, 1),
  (15, 1),
  (8, 4),
  (18, 1),
  (2, 3),
  (22, 1),
  (25, 4),
  (5, 2),
  (19, 4),
  (13, 1),
  (21, 4),
  (24, 4)],
 'MATH11231': [(22, 3), (1, 4), (23, 4)],
 'MATH08077': [(4, 1),
  (4, 4),
  (0, 0),
  (11, 1),
  (13, 2),
  (4, 3),
  (11, 3),
  (6, 1),
  (2, 3),
  (5, 0),
  (11, 2),
  (25, 3),
  (17, 3),
  (17, 4),
  (10, 2)],
 'MATH08057': [(15, 1),
  (27, 3),
  (10, 0),
  (19, 4),
  (12, 4),
  (17, 3),
  (0, 3),
  (11, 0),
  (15, 0),
  (6, 4),
  (7, 2),
  (17, 1),
  (27, 4),
  (20, 4),
  (11, 1),
  (15, 4),
  (22, 0),
  (11, 3),
  (17, 4),
  (9, 3),
  (26, 1),
  (13, 4),
  (16, 2),
  (16, 3),
  (13, 4),
  (24, 0),
  (22, 1),
  (26, 0),
  (4, 1),
  (13, 2),
  (7, 4),
  (27, 1),
  (9, 3),
  (5, 1),
  (5, 1),
  (1, 0),
  (7, 2),
  (24, 2),
  (18, 0),
  (1

In [11]:
# Create a list of all possible room and weekday combinations.
room_timetable = [(room, weekday) for room in range(rooms_num) for weekday in range(weekdays_num)]

# Initialize a dictionary to store course arrangements for each room and weekday.
room_timetable_dict = {key: [] for key in room_timetable}

# Populate the dictionary with course data from a pre-existing timetable.
for course in pre_timetable:
    arrangement_list = pre_timetable[course]
    for room_date in arrangement_list:
        # Each course is paired with its duration and stored under the corresponding room and date.
        course_time = (course, hours_per_course[course])
        room_timetable_dict[room_date].append(course_time)

# Iterate over each room and weekday combination to display the scheduled courses.
for rt in room_timetable:
    print(rooms[rt[0]], ":")
    # Initialize a counter to calculate start times based on course durations.
    k = 0
    for one_class in room_timetable_dict[rt]:
        # Format the course code and the specific day.
        first_part = "  - Course Code: " + one_class[0] + ", " + "Day " + str(rt[1] + 1) + ", "
        # Calculate start and end times based on course duration, starting at 9 AM.
        second_part = [9 + k, 9 + k + one_class[1]]
        for j in range(2):
            # Convert 24-hour time format to 12-hour format with AM/PM.
            if second_part[j] < 12:
                second_part[j] = str(second_part[j]) + "AM"
            elif second_part[j] == 12:
                second_part[j] = "12PM"
            else:
                second_part[j] = str(second_part[j] % 12) + "PM"
        # Append the duration of the course in hours.
        third_part = "(" + str(one_class[1]) + "h)"
        # Increment the start time by the duration of the current course for the next course.
        k += one_class[1]
        # Print the complete timetable entry for each course.
        print(first_part + second_part[0] + "-" + second_part[1] + third_part)


JBB_G.69 :
  - Course Code: MATH08077, Day 1, 9AM-11AM(2h)
  - Course Code: MATH10068+MATH08068+MATH10079, Day 1, 11AM-1PM(2h)
  - Course Code: MATH10072, Day 1, 1PM-2PM(1h)
  - Course Code: MATH11154, Day 1, 2PM-3PM(1h)
  - Course Code: MATH08074 + MATH10095, Day 1, 3PM-4PM(1h)
  - Course Code: MATH07003 + MATH11154, Day 1, 4PM-5PM(1h)
  - Course Code: MATH11199A, Day 1, 5PM-6PM(1h)
JBB_G.69 :
  - Course Code: MATH10066, Day 2, 9AM-11AM(2h)
  - Course Code: MATH10072, Day 2, 11AM-12PM(1h)
  - Course Code: MATH08074, Day 2, 12PM-1PM(1h)
  - Course Code: MATH10076, Day 2, 1PM-2PM(1h)
  - Course Code: MATH08063, Day 2, 2PM-3PM(1h)
  - Course Code: MATH10017 + MATH10065, Day 2, 3PM-4PM(1h)
JBB_G.69 :
  - Course Code: MATH10098, Day 3, 9AM-11AM(2h)
  - Course Code: MATH08074, Day 3, 11AM-12PM(1h)
  - Course Code: MATH08063, Day 3, 12PM-1PM(1h)
  - Course Code: MATH11176, Day 3, 1PM-2PM(1h)
  - Course Code: MATH10017 + MATH10065, Day 3, 2PM-3PM(1h)
  - Course Code: MATH08074 + MATH10095, Da